# Uniswap V2 Tutorial

This tutorial shows how to use Uniswap V2 with Dexsnake. Let's start by connecting to the blockchain (here, we use [Base](https://www.base.org/) for its relatively low transaction fees).

In [1]:
import os

from web3 import Web3


# the code below assumes you have configured the required environment variables
account = os.getenv("WEB3_PUBLIC_KEY")
private_key = os.getenv("WEB3_PRIVATE_KEY")
provider_url = os.getenv("WEB3_PROVIDER_URL")

web3 = Web3(Web3.HTTPProvider(provider_url))

Now, let's import `UniswapV2Factory` and find the pair with USDC and WETH.

In [2]:
from dexsnake.uniswap_v2 import UniswapV2Factory


factory = UniswapV2Factory(web3)

USDC_address = "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913"
WETH_address = "0x4200000000000000000000000000000000000006"

pair_address = factory.get_pair(USDC_address, WETH_address)

print(pair_address)

0x88A43bbDF9D098eEC7bCEda4e2494615dfD9bB9C


Now, let's initialize the pair object and fetch the pair's reserves and price.

In [3]:
from dexsnake.uniswap_v2 import UniswapV2Pair


pair = UniswapV2Pair(web3, pair_address)

reserves_0, reserves_1 = pair.get_reserves()

price = pair.get_price()

print(f"Reserves of {pair.token_0.symbol}: {reserves_0}")
print(f"Reserves of {pair.token_1.symbol}: {reserves_1}")
print(f"Price of 1 {pair.token_0.symbol}: {price} {pair.token_1.symbol}")

Reserves of WETH: 440.61604517871905
Reserves of USDC: 1139872.568688
Price of 1 WETH: 2586.9974122836456 USDC


Now that we know the pair has good liquidity at a good price, let's use the router to swap one USDC for WETH. Note that before you can make the trade, you must approve the router contract to spend your USDC.

In [4]:
from dexsnake.uniswap_v2 import UniswapV2Router


router = UniswapV2Router(web3)

tx_receipt_approval = pair.token_1.approve(router.contract.address, 1, account, private_key)

tx_receipt_swap = router.swap_exact_tokens_for_tokens(
    amount_in=1,
    amount_out_min=1 / price * 0.99,  # max 1% slippage
    path=[USDC_address, WETH_address],
    to=account,
    account=account,
    private_key=private_key,
)

tx_receipt_swap

AttributeDict({'blockHash': HexBytes('0x29e248dd6d22b55343c5a9d6405d142093629b7be675cab5e7ae857f11ec23f0'),
 'blockNumber': 18640731,
 'contractAddress': None,
 'cumulativeGasUsed': 4174043,
 'effectiveGasPrice': 5355749,
 'from': '0x011b7DD4cAA8E48FbBaef548dB349dfB3D09aA3F',
 'gasUsed': 110559,
 'l1BaseFeeScalar': '0x8dd',
 'l1BlobBaseFee': '0x1',
 'l1BlobBaseFeeScalar': '0x101c12',
 'l1Fee': '0x224dcb614',
 'l1GasPrice': '0x686a9e2a',
 'l1GasUsed': '0x90c',
 'logs': [AttributeDict({'address': '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000011b7dd4caa8e48fbbaef548db349dfb3d09aa3f'),
    HexBytes('0x00000000000000000000000088a43bbdf9d098eec7bceda4e2494615dfd9bb9c')],
   'data': HexBytes('0x00000000000000000000000000000000000000000000000000000000000f4240'),
   'blockNumber': 18640731,
   'transactionHash': HexBytes('0x35b8a3e88b00143a79a1c8883938ce37f4ac23

You can inspect the transaction on the block explorer: https://basescan.org/tx/0x35b8a3e88b00143a79a1c8883938ce37f4ac23a848c774f7604520daea003b6c.